# Explore the outputted model files for an example model 

https://machinelearningmastery.com/save-load-keras-deep-learning-models/

This notebook follows thru the example presented above and digs into the viewing/accessing the outputted model files generated into HDF5 and JSON files.

In [12]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import tensorflow
import numpy
import os

In [15]:
print(numpy.__version__)
print(tensorflow.__version__)
#!pip install h5pyViewer
# https://stackoverflow.com/questions/51173580/how-to-read-keras-model-weights-without-a-model

1.16.4
1.14.0


In [16]:
# fix random seed for reproducibility
numpy.random.seed(7)

# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, epochs=150, batch_size=10, verbose=0)

W0823 12:56:02.043327  9104 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 12:56:02.114330  9104 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 12:56:02.138327  9104 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0823 12:56:02.199330  9104 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0823 12:56:02.305327  9104 deprecation_wrapper.py:119] From C:\ProgramData\Anac

In [17]:
# evaluate the model
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 77.86%


In [18]:
# serialize model to JSON
model_json = model.to_json()
with open("pima_model.json", "w") as json_file:
    json_file.write(model_json)

In [19]:
# serialize weights to HDF5
model.save_weights("pima_model.h5")
print("Saved model to disk")

Saved model to disk


In [24]:
# later...

# load json and create model
json_file = open('pima_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("pima_model.h5")
print("Loaded model from disk")

# Look at HDF5 model parameter structure

In [20]:
import h5py
f=h5py.File('pima_model.h5', 'r') #import the model file with 'read' permissions

print(list(f.keys()))
#dset = f['dense_1']

['dense_1', 'dense_2', 'dense_3']


In [21]:
d=f['dense_1']
print(f.attrs.items())

ItemsViewHDF5(<Attributes of HDF5 object at 1586776884416>)


In [26]:
print(f.attrs.items())
#if len(f.attrs.items()):
   # print("{} contains: ".format(weight_file_path))
    #print("Root attributes:")
    #print("  f.attrs.items(): ")
    
for key, value in f.attrs.items():
    #print(key, "  ", value)
    print("  {}: {}".format(key, value))

    if len(f.items())==0:
        print("  Terminate # len(f.items())==0: ")

ItemsViewHDF5(<Attributes of HDF5 object at 1586776885384>)
  backend: b'tensorflow'
  keras_version: b'2.2.5'
  layer_names: [b'dense_1' b'dense_2' b'dense_3']


In [27]:
for layer, g in f.items():            
            print("  {}".format(layer))
            print("    g.attrs.items(): Attributes:")
            for key, value in g.attrs.items():
                print("      {}: {}".format(key, value))

            print("    Dataset:")
            for p_name in g.keys():
                param = g[p_name]
                subkeys = param.keys()
                print("    Dataset: param.keys():")
                for k_name in param.keys():
                    print("      {}/{}: {}".format(p_name, k_name, param.get(k_name)[:]))

  dense_1
    g.attrs.items(): Attributes:
      weight_names: [b'dense_1/kernel:0' b'dense_1/bias:0']
    Dataset:
    Dataset: param.keys():
      dense_1/bias:0: [-1.7412677   0.          1.207784   -0.16921678  1.0718697   1.2052598
 -0.9765404   1.1325557   1.41263     0.49467513 -1.5568869   0.        ]
      dense_1/kernel:0: [[ 0.23350556  0.01882353 -0.3646723  -0.30434564 -0.15309867 -0.31964445
   0.1690051  -0.33440226 -0.41950876 -0.5037846   0.17060773 -0.035793  ]
 [ 0.09010825 -0.03511642  0.02143405  0.08531553  0.015311   -0.04998917
  -0.11017638  0.06727614  0.04060419 -0.1062217   0.12676387 -0.01599389]
 [-0.07434946 -0.00493143  0.09144411  0.17805102  0.10590395  0.18729377
  -0.00898929  0.05190195  0.07317782  0.17975095 -0.07113796 -0.04335348]
 [ 0.05266909 -0.01141974  0.07563986  0.1091879  -0.05200028 -0.17930453
   0.07660628 -0.01942675  0.0621646   0.01402787 -0.05933642  0.02202759]
 [-0.03714687 -0.00778842 -0.04004614  0.01483318 -0.0241155   0.0449

In [25]:
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

score = loaded_model.evaluate(X, Y, verbose=0)

print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

acc: 77.86%
